<a href="https://colab.research.google.com/github/abirpy/NLP-Projects/blob/master/distilbert_base_uncased_finetuned_emotion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets transformers[sentencepiece]

In [ ]:
!pip install huggingface

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

from datasets import load_dataset

dataset = load_dataset("emotion")

In [ ]:
dataset

In [5]:
import pandas as pd
import numpy as np

In [ ]:
arr = pd.Series(dataset['train']['label'])
arr.value_counts()

In [ ]:
label_names = dataset['train'].features['label'].names
label_names

In [8]:
id2label = {k:v for k,v in enumerate(label_names)}
label2id = {v:k for k,v in id2label.items()}

In [ ]:
id2label

In [ ]:
from transformers import AutoTokenizer, DataCollatorWithPadding

checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [ ]:
def tokenize_function(example):
    return tokenizer(example["text"], truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
tokenized_datasets['train']

In [ ]:
tokenized_datasets = tokenized_datasets.remove_columns(['text'])
tokenized_datasets["train"].column_names

In [ ]:
tokenized_datasets['train'][0]

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    checkpoint,
    id2label = id2label,
    label2id = label2id,
)

In [ ]:
model.config.num_labels

In [17]:
from transformers import TrainingArguments

args = TrainingArguments(
    "distilbert-base-uncased-finetuned-emotion",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    per_device_train_batch_size = 64,
    per_device_eval_batch_size = 64,
    push_to_hub=True,
)

In [ ]:
!huggingface-cli login

In [19]:
!git config --global credential.helper store

In [ ]:
!apt install git-lfs

In [ ]:
!pip install bert_score

In [26]:
from datasets import load_metric
def compute_metrics(eval_preds):
    metric = load_metric("accuracy")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

In [ ]:
trainer.push_to_hub(commit_message="Training complete")

In [ ]:
from transformers import pipeline
mood = pipeline('sentiment-analysis', model='Tahsin/distilbert-base-uncased-finetuned-emotion')
mood("I love my country")